In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Box
import ipywidgets as widgets
from IPython.display import display ,Javascript
import geopandas as gpd
import itertools
from datetime import datetime
import os
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# i'd like to add a year slider and rep slider with some modif to the PPt

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='None',
                    width='100%')
def run_below(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

def run_below2(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
def run_model( chunk , cores ):
    import subprocess, os
    n=0
    for i in chunk :
        n=n+1
        head = '#!/bin/sh\n' + \
            '#SBATCH --ntasks=%s\n' %cores + \
            '#SBATCH --nodes=1\n' + \
            '#SBATCH --ntasks-per-node=%s\n'%cores + \
            '#SBATCH --account=snap\n' + \
            '#SBATCH --mail-type=FAIL\n' + \
            '#SBATCH --mail-user=jschroder@alaska.edu\n' + \
            '#SBATCH -p main\n\n' \
            'source ~/miniconda2/envs/ALFPP/bin/activate'

        slurm_fn = 'PP_{}_{}.slurm'.format(datetime.now().strftime("%Y-%m-%d-%H-%M"),n)
        with open( slurm_fn, 'w' ) as f:
            f.writelines( head + '\n' )
            f.writelines(i)

        subprocess.call([ 'sbatch', slurm_fn ])
    return 1
            
def chunkify(lst,n):
    return [ lst[i::n] for i in range(n) ]

def cmd_builder(script,model,base_path,shapefile_path,field,name,output_path):
    return 'python ' + script + ' -p ' + \
                    str(os.path.join(base_path,model)) + ' -shp ' + str(shapefile_path) + ' -field ' + str(field) + ' -name ' + \
                    str(name) + ' -o ' + str(output_path)

metrics = [ 'veg_counts','avg_fire_size','number_of_fires','all_fire_sizes','total_area_burned' ]
rcp45_models  = ['GFDL-CM3', 'GISS-E2-R', 'NCAR-CCSM4', 'IPSL-CM5A-LR','MRI-CGCM3']
layout = Layout(flex='0 1 auto', width='auto')

button = widgets.Button(description="Click before continuing")
button_run = widgets.Button(description="Run!")
cores = widgets.IntSlider(
    value=32,
    min=1,
    max=32,
    step=1,
    description='Cores',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
nodes = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Nodes',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
years = widgets.IntRangeSlider(
    value=[1900, 2100],
    min=1900,
    max=2100,
    step=1,
    description='Years',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white',
    color='black'
)
historical_path = widgets.Text(
    value='/Data/Base_Data/ALFRESCO/ALFRESCO_Master_Dataset/ALFRESCO_Model_Input_Datasets/AK_CAN_Inputs/Fire'
,
    description='Historical Path:',
    disabled=False,
    layout=layout
)
base_path = widgets.Text(
    value='/atlas_scratch/apbennett/Calibration/HighCalib/FMO_Calibrated',
    description='Base path:',
    disabled=False,
    layout=layout
)
script_path = widgets.Text(
    value='/workspace/Shared/Users/jschroder/Github/ALFRESCO/PP_launcher.py'
,
    description='Launcher Path:',
    disabled=False,
    layout=layout
)
CSV = widgets.Checkbox(
    value=False,
    description="CSV",
    disabled=False
)
output_path = widgets.Text(
    value="/atlas_scratch/jschroder/ALF_outputs/ALF_PP_{}".format(datetime.now().strftime("%Y-%m-%d-%H-%M")
),
    description='Output Path :',
    disabled=False,
    layout=layout
)
metrics = widgets.SelectMultiple(
    options=metrics,
    value=[],
    description='Metrics',
    disabled=False
)
model = widgets.SelectMultiple(
    options=rcp45_models,
    value=[],
    description='RCP45',
    disabled=False
)
scenario = widgets.SelectMultiple(
    options=['rcp45','rcp60','rcp85'],
    value=[],
    description='Scenarios',
    disabled=False
)
FMO = widgets.SelectMultiple(
    options=['NoFMO','Normal','AltFMO','PreFMO'],
    value=['Normal'],
    description='FMPO',
    disabled=False
)
shapefile_path = widgets.Text(
    value='/workspace/Shared/Users/jschroder/Alec_Alfresco_PP_improvements/Data/Shapefile/Boreal_Tundra_CombinedDomains.shp',
    description='Shapefile Path:',
    disabled=False,
    layout=layout
)
hist = widgets.Checkbox(
    value=False,
    description='Historical',
    disabled=False
)

**Set the reference folders :**

In [23]:
display(script_path)
display(base_path)
display(historical_path)
display(output_path)
display(shapefile_path)

In [24]:
if not os.path.exists( output_path.value ):
    os.makedirs( output_path.value )


**Set numerical parameters :**

In [9]:
display(nodes)
display(cores)
display(years)

**Metrics to be computed :**

In [10]:
display(metrics)

**Models, Scenarios and FMO options to be computed :**

In [11]:

box = Box(children=[model,scenario,FMO], layout=box_layout)
display(box)



**Optional computation :**

In [12]:
a=itertools.product(model.value,scenario.value,FMO.value)
l = [ "_".join(i) for i in a]
display(hist)
display(CSV)


In [13]:

button.on_click(run_below)
display(button)

**Elements to be computed :**

In [25]:
a=itertools.product(model.value,scenario.value,FMO.value)
l = [ "_".join(i) for i in a]
l= [i.replace('_Normal','') for i in l if "_Normal" in i]
for i in l :
    print(i)

GFDL-CM3_rcp85
GISS-E2-R_rcp85
NCAR-CCSM4_rcp85
IPSL-CM5A-LR_rcp85
MRI-CGCM3_rcp85


In [26]:
if len(l)>0 :
    shapefile = gpd.read_file(shapefile_path.value)
    print("Shapefile's attribute table :\n")
    print(shapefile.head())

#     shapefile.plot()
#     plt.title("Shapefile's preview")
#     plt.axis('off')

    
else : print("ERROR : at least one model, scenarios and FMPO need to be selected")

Shapefile's attribute table :

    Id    Name                                           geometry
0  1.0  Boreal  (POLYGON ((-251965.7485199124 1818649.32538744...
1  2.0  Tundra  (POLYGON ((-407522.7285999991 972567.568700000...


**What is the unique integral identifier?**

In [27]:
if len(l)>0 :
    id_field = widgets.Dropdown(
        options=list(["None"]) + list(shapefile.head()),
        value="None",
        description='ID field :',
        disabled=False)
    

    display(id_field)
else : print("ERROR : at least one model, scenarios and FMPO need to be selected")

**What field contain the domain's names?**

In [28]:
if len(l)>0 :
    name = widgets.Dropdown(
        options=list(["None"]) + list(shapefile.head()),
        value="None",
        description='Name field :',
        disabled=False)


    display(name)
else : print("ERROR : at least one model, scenarios and FMPO need to be selected")

In [29]:
button_run.on_click(run_below2)
display(button_run)

<IPython.core.display.Javascript object>

In [30]:
if len(l)>0 :
    if ((id_field.value!='None') & (name.value!='None')) :
        cmd =[cmd_builder(script_path.value,i,base_path.value,shapefile_path.value,id_field.value,name.value,output_path.value)+'\n' for i in l]
        chunk = chunkify(cmd,nodes.value)
        run_model(chunk,nodes.value)

    else : print("ERROR : Please select an ID field, a name field and click Run")